In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle
from IPython.display import clear_output
import stata_setup
stata_setup.config("C:/Program Files/Stata17/", "mp")
from pystata import stata

In [ ]:
df = pd.read_csv(r"fcfe_1.csv")
df["Date"] = pd.to_datetime(df["Date"], format='%Y-%m-%d')

In [ ]:
def stata_exuberance(group, series):
    stata.pdataframe_to_data(group[["Date",series]], force=True)

    stata.run(f'''
    gen Date1 = _n
    tsset Date1
    radf {series}, prefix(_t) criterion(aic) boot(200)

              ''',quietly = True)
    return stata.pdataframe_from_data(), stata.get_return()["r(radfstats)"]
    

In [ ]:
def test_exuberance(df,series): #df is the main original dataframe, series is FCFE or dividends
    # INPUT A SERIES WITHOUT SPACES OR OTHERWISE THE OUTPUT RETURNED FROM STATA WILL NOT BE READ PROPERLY
        
    
    df["Price_Exuberance"] = 0
    df[f"{series}_Exuberance"] = 0
    # column in original dataframe that can be used to run panel regressions later
    
    i = 0
    length = len(df["Adjusted_LPERMNO"].unique())
    errors = 0
    
    for permno, group in df.groupby("Adjusted_LPERMNO"): 
        
        clear_output(wait=True)
        try:
            # define initial dates so that for dividends the error raised can still work with df.loc
            # and similarly, for any series that raise an error in general if an error occurs during stat_exuberance.
            # otherwise you end up looking at the 'initial dates' from the previous loop!
            initial_dates = list(group["Date"])
            
            df2, test = stata_exuberance(group,"Price")
            test_statistic=test[2][0] # 3rd row first value
            crit_value = test[2][2] # 3rd row 3rd value, 95% cr

            initial_dates = list(df2[df2["_tExceeding"]>1]["Date"]) # rolling window initial dates
            # Stata's null values are read as an integer overflow when ported back to Python
            exuberance_dates = list(df2[df2["_tExceeding"]==1]["Date"])

            df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(initial_dates)),"Price_Exuberance"] += 2
            #drop these ^ later
            
            if test_statistic>crit_value:
                df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(exuberance_dates)),"Price_Exuberance"] += 1
                
                # now for the fundamental
                
                initial_dates = list(group["Date"])
            
                df2, test = stata_exuberance(group,series)
                test_statistic2=test[2][0] # 3rd row first value
                crit_value2 = test[2][2] # 3rd row 3rd value, 95% cr

                initial_dates = list(df2[df2["_tExceeding"]>1]["Date"]) # rolling window initial dates
                # Stata's null values are read as an integer overflow when ported back to Python
                exuberance_dates = list(df2[df2["_tExceeding"]==1]["Date"])

                df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(initial_dates)),f"{series}_Exuberance"] += 2
                
                if test_statistic2>crit_value2:
                    df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(exuberance_dates)),f"{series}_Exuberance"] += 1
                        

            # can keep usable but non-exuberant dates as zero in the original df's series exuberance column
            # which is zero by default

        except:
            errors+=1
            df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(initial_dates)),"Price_Exuberance"] += 3
           
        i+=1
        print(i,"/",length) # progress counter
        
        print(errors)

So, if the series is 'Price', then the Price_Exuberance column will be 3 if an error occurred, 2 if the observation was used in the initial rolling window, 1 if there was exuberance and 0 if no exuberance.

In [ ]:
i = 0
length = len(df["Adjusted_LPERMNO"].unique())
for permno, group in df.groupby("Adjusted_LPERMNO"):
    clear_output(wait=True)
    lowest = group["FCFE"].min() # REMEMBER TO ADJUST THIS TOO
    if lowest <0:
        df.loc[df["Adjusted_LPERMNO"]==permno,"FCFE"]+= (1-lowest)
    i+=1
    print(i,"/",length)
        

In [ ]:
test_exuberance(df,"FCFE")
df.to_csv("netincome_tested.csv",index=False)
df=df[df["Price_Exuberance"]<=1].reset_index(drop=True)
df2 = df.groupby("Date",as_index=False)["Price_Exuberance"].sum()

In [ ]:
dates = pd.date_range('30/06/1973','30/06/2005',freq = 'Q')
exuberance = pd.DataFrame({"Date":dates,"Total Market Cap":np.zeros(len(dates))})

# getting total market cap
for date, group in df.groupby("Date"):
    exuberance.loc[exuberance.Date ==date,"Total Market Cap"] = group["Market Cap"].sum() 
    # total nasdaq market cap at this date
    
# creating sectoral columns to record exuberance
for i in df["Sector"].unique():
    exuberance[i]=np.zeros(len(dates))
    # exuberance[i+"_total"]=np.zeros(len(dates))
    
# getting exuberant market cap for sectors
for date_sector, group in df.groupby(["Date","Sector"]):
    exuberant_market_cap = group.loc[group["Price_Exuberance"]==1,"Market Cap"].sum()
    
    # ^Price Exuberance!!!
    
    # because we group by both date and sector, the iterator is a list containing the date and sector
    
    exuberance.loc[exuberance.Date ==date_sector[0],date_sector[1]] = exuberant_market_cap
    # edit corresponding sectoral column of exuberance
    
exuberance["Total Exuberance Market Cap"] = exuberance[list(df["Sector"].unique())].sum(axis=1)
    


# note that this is an underestimate of exuberance because we can't capture exuberance for all series such as IPO exuberance as well

In [ ]:
sector_dict = {"45":"Information Technology","50":"Communication Services","20":"Industrials",
              "25":"Consumer Discretionary","35":"Healthcare","30":"Consumer Staples","15":"Materials",
              "55":"Utilities","10":"Energy","60":"Real Estate","40":"Financials","Other":"Other"}

palette = cycle(px.colors.qualitative.Pastel+px.colors.qualitative.Bold)
bars =  []
for sector in df["Sector"].unique():
    bars.append(
        go.Bar(name = sector_dict[sector], x = exuberance["Date"], 
               y = exuberance[sector], marker_color = next(palette) 
                      )
    )

fig = go.Figure(data = bars)

fig.update_layout(barmode='stack')

#     fig.add_trace(
#         go.Bar(name = "Total Market Cap", x = exuberance["Date"], 
#                    y = exuberance["Total Market Cap"], marker_color = next(palette),opacity = 0.5)
#         )

fig.update_layout(
    title="NASDAQ Exuberance by Sector, 1973-2005, measured in market cap",
    xaxis_title="Date",
    yaxis_title="Real Market Cap of stocks in exuberance")
fig.show()

# need to use bar chart because you can't stack histograms easily to show sector composition.

Exuberance testing functions.

In [ ]:
def create_exuberance_df(df):
    dates = pd.date_range('30/06/1973','30/06/2005',freq = 'Q')
    exuberance = pd.DataFrame({"Date":dates,"Frequency":np.zeros(len(dates)),"Total":np.zeros(len(dates))})

    for i in df["Sector"].unique():
        exuberance[i]=np.zeros(len(dates))
        exuberance[i+"_total"]=np.zeros(len(dates))
        
    return(exuberance)
    

In [ ]:
def stata_exuberance(group,series):

    stata.pdataframe_to_data(group[["Date",series]], force=True)

    stata.run(f'''
    gen Date1 = _n
    tsset Date1
    radf {series}, prefix(_t)
              ''',quietly = True)

    return stata.pdataframe_from_data()

In [ ]:
def test_exuberance(df,series): # df is the main original dataframe, series is Price or Sales or Revenue etc.
    # INPUT A SERIES WITHOUT SPACES OR OTHERWISE THE OUTPUT RETURNED FROM STATA WILL NOT BE READ PROPERLY
        
    exuberance = create_exuberance_df(df)
    # dataframe that tracks total exuberance over time
    
    df[f"{series}_Exuberance"] = 0
    # column in original dataframe that can be used to run panel regressions later
    
    i = 1
    length = len(df["Adjusted_LPERMNO"].unique())
    errors = 0
    
    for permno, group in df.groupby("Adjusted_LPERMNO"): 
        
        clear_output(wait=True)
        try:
            
            if series =="RealDividends" or series == "LogRealDividends":
                if (group['Dividends'] == 0).all():
                    raise ValueError

            df2 = stata_exuberance(group,series)

            initial_dates = list(df2[df2["_tExceeding"]>1]["Date"])
            # Stata's null values are read as an integer overflow when ported back to Python
            exuberance_dates = list(df2[df2["_tExceeding"]==1]["Date"])

            total_dates = exuberance_dates + list(df2[df2["_tExceeding"]==0]["Date"])
            # all the dates used in the regression, excluding initial regression dates

            df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(initial_dates)),f"{series}_Exuberance"] += 2
            # drop these ^ later
            df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(exuberance_dates)),f"{series}_Exuberance"] += 1

            # can keep usable but non-exuberant dates as zero in the original df's series exuberance column
            # which is zero by default

            exuberance.loc[exuberance.Date.isin(total_dates),"Total"] +=1
            exuberance.loc[exuberance.Date.isin(exuberance_dates), "Frequency"] += 1
            corresponding_sector = group["Sector"].iloc[0]
            exuberance.loc[exuberance.Date.isin(exuberance_dates), corresponding_sector] += 1
            exuberance.loc[exuberance.Date.isin(total_dates), corresponding_sector+"_total"] += 1
            # sectoral information

        except:
            errors+=1
            df.loc[(df["Adjusted_LPERMNO"] ==permno)&(df["Date"].isin(initial_dates)),f"{series}_Exuberance"] += 3
           
        i+=1
        print(i,"/",length) # progress counter
        print(errors)
        
    return exuberance
    

So, if the series is 'Price', then the Price_Exuberance column will be 3 if an error occurred, 2 if the observation was used in the initial rolling window, 1 if there was exuberance and 0 if no exuberance.

In [ ]:
realprice_exuberance = test_exuberance(df,"RealPrice")
realsales_exuberance = test_exuberance(df,"RealSales")
realnetincome_exuberance = test_exuberance(df,"RealnetIncome") 
logrealprice_exuberance = test_exuberance(df,"LogRealPrice")
realdiv_exuberance = test_exuberance(df,"RealDividends") 
logrealdiv_exuberance = test_exuberance(df,"LogRealDividends")

# stata doesn't like spaces in variable names!
# reading a variable back from Stata would remove the space and mess up my code
# test dividends

In [ ]:
def sectoral_chart(exuberance): #exuberance being the class of dataframe generated above
    
    sector_dict = {"45":"Information Technology","50":"Communication Services","20":"Industrials",
                  "25":"Consumer Discretionary","35":"Healthcare","30":"Consumer Staples","15":"Materials",
                  "55":"Utilities","10":"Energy","60":"Real Estate","40":"Financials","Other":"Other"
                 }
    
    palette = cycle(px.colors.qualitative.Pastel+px.colors.qualitative.Bold)
    bars =  []
    for sector in df["Sector"].unique():
        bars.append(
            go.Bar(name = sector_dict[sector], x = exuberance["Date"], 
                   y = exuberance[sector], marker_color = next(palette) 
                          )
        )

    fig = go.Figure(data = bars)

    fig.update_layout(barmode='stack')
    
#     fig.add_trace(
#         go.Bar(name = "Total Available", x = exuberance["Date"], 
#                    y = exuberance["Total"], marker_color = next(palette),opacity = 0.5)
#         )

    fig.update_layout(
        title="NASDAQ Exuberance by Sector, 1973-2005",
        xaxis_title="Date",
        yaxis_title="Number of stocks in exuberance")
    return fig

# need to use bar chart because you can't stack histograms easily to show sector composition.

In [ ]:
realprice_graph = sectoral_chart(realprice_exuberance)

realprice_graph.update_layout(
    title="NASDAQ Real Stock Price Exuberance by Sector, 1973-2005",
    xaxis_title="Date",
    yaxis_title="Number of stocks in exuberance")
realprice_graph.show()
# need to weight by market cap

In [ ]:
realprice_graph = sectoral_chart(realprice_exuberance)

realprice_graph.update_layout(
    title="NASDAQ Real Stock Price Exuberance by Sector, 1973-2005",
    xaxis_title="Date",
    yaxis_title="Number of stocks in exuberance")

realprice_graph.write_image("Real Stock Price Exuberance.png")

realsales_graph = sectoral_chart(realsales_exuberance)

realsales_graph.update_layout(
    title="NASDAQ Real Sales Exuberance by Sector, 1973-2005",
    xaxis_title="Date",
    yaxis_title="Number of stocks with exuberant sales")

realsales_graph.write_image("Real Sales Exuberance.png")

#

realincome_graph = sectoral_chart(realincome_exuberance)

realincome_graph.update_layout(
    title="NASDAQ Real Net Income Exuberance by Sector, 1973-2005",
    xaxis_title="Date",
    yaxis_title="Number of stocks with exuberant net income")

realincome_graph.write_image("Real Net Income Exuberance.png")

#

realdiv_graph = sectoral_chart(realdiv_exuberance)

realdiv_graph.update_layout(
    title="NASDAQ Real Dividends Exuberance by Sector, 1973-2005",
    xaxis_title="Date",
    yaxis_title="Number of stocks with exuberant dividends")

realdiv_graph.write_image("Real Dividends Exuberance.png")

#
logrealprice_graph = sectoral_chart(logrealprice_exuberance)

logrealprice_graph.update_layout(
    title="NASDAQ Log Real Stock Price Exuberance by Sector, 1973-2005",
    xaxis_title="Date",
    yaxis_title="Number of stocks in exuberance")

logrealprice_graph.write_image("Log Real Stock Price Exuberance.png")

#
logrealdiv_graph = sectoral_chart(logrealdiv_exuberance)

logrealdiv_graph.update_layout(
    title="NASDAQ Log Real Dividends Exuberance by Sector, 1973-2005",
    xaxis_title="Date",
    yaxis_title="Number of stocks with exuberant dividends")

logrealdiv_graph.write_image("Log Real Dividends Exuberance.png")
#
df.to_csv("compfund_exuberance.csv",index=False)

In [ ]:
df[df.LPERMNO ==10005][["Price","LogRealPrice","RealPrice_Exuberance","LogRealPrice_Exuberance"]]
# some cases, you have the price not changing for several continuous quarters.
# unclear as to whether this is poor data or if the stock is just highly illiquid/unvolatile.
# however, once you adjust for inflation, that introduces a downtrend in the real price.
# and then when you take logs of the downtrend, it becomes negative.
# when you test a negative explosive series, you would then find evidence of exuberance when it isn't really there.
# phillips did not encounter this problem because his stock index took relatively high values.

# so, best not to take logs.